# init

## environment preparation
- You may find some errors on %run command, but should be fine as far as data download is done 

In [1]:
# clone repo
!git clone https://github.com/NeuromatchAcademy/course-content

Cloning into 'course-content'...
remote: Enumerating objects: 54444, done.
remote: Counting objects: 100% (5358/5358), done.
remote: Compressing objects: 100% (1953/1953), done.
remote: Total 54444 (delta 3346), reused 5242 (delta 3277), pack-reused 49086
Receiving objects: 100% (54444/54444), 2.94 GiB | 14.24 MiB/s, done.
Resolving deltas: 100% (34007/34007), done.
Updating files: 100% (804/804), done.


In [1]:
# run the notebook for loading dataset
%run ./course-content/projects/ECoG/load_ECoG_motor_imagery.ipynb

/home/s2140401/anaconda3/envs/bda/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


  ERROR: Command errored out with exit status 1:
   command: /home/s2140401/anaconda3/envs/bda/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-welq1x8j/matplotlib_77fa7dab082d4a23b5ef84f0e84cb43d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-welq1x8j/matplotlib_77fa7dab082d4a23b5ef84f0e84cb43d/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-pz9zlm0l
       cwd: /tmp/pip-install-welq1x8j/matplotlib_77fa7dab082d4a23b5ef84f0e84cb43d/
  Complete output (813 lines):
  
  Edit setup.cfg to change the build options; suppress output with --quiet.
  
  BUILDING MATPLOTLIB
    matplotlib: yes [3.4.3]
        python: yes [3.10.4 (main, Mar 31 2022, 08:41:55) [GCC 7.5.0]]
      platform: yes [linux]


ModuleNotFoundError: No module named 'nimare'

ModuleNotFoundError: No module named 'nimare'

## modules / variables

In [10]:
import os
import pickle
from pprint import pprint

import numpy as np
import pandas as pd
import pandas_profiling as pdp
import plotly.express as px
from tqdm import tqdm, trange

from processing.ECoG import default_ECoG_processor, electrode_mapper
from processing.ECoG import dat_df_generator, convert_to_batches

In [2]:
FNAME_ECOG = "motor_imagery.npz"

## ECoG
- motor imagery as an example

### load

In [11]:
# list(np.load(FNAME_ECOG, allow_pickle=True).keys())

In [3]:
all = np.load(FNAME_ECOG, allow_pickle=True)
pprint(list(all.keys()))
alldat = all['dat']
pprint(alldat.shape)#(subject, experiment: 0 is real, 1 is imagery)
pprint(alldat[0][0].keys())

['dat']
(7, 2)
dict_keys(['t_off', 'stim_id', 't_on', 'srate', 'V', 'scale_uv', 'locs', 'hemisphere', 'lobe', 'gyrus', 'Brodmann_Area'])


### preprocessing

#### preprocessing in notebook

In [10]:
# # one recording
# dat1 = alldat[0][0]
# print(dat1.keys())
# print(dat1["srate"])
# print(dat1["t_on"][0:3])
# print(dat1["stim_id"][0:3])
# print(dat1["locs"].shape)

In [20]:
# preprocessing in notebook
Vs = {}
for subject in trange(len(alldat)):
    Vs[f"subject_{subject}"] = {}
    for experiment in range(len(alldat[0])):
        V = default_ECoG_processor(alldat[subject][experiment])
        Vs[f"subject_{subject}"][f"experiment_{experiment}"] = V
pprint(Vs.keys())
pprint(Vs["subject_0"].keys())
pprint(Vs["subject_0"]["experiment_1"].keys())
pprint(Vs["subject_0"]["experiment_1"]["stimuli_11"].shape)

100%|██████████| 7/7 [00:54<00:00,  7.82s/it]

dict_keys(['subject_0', 'subject_1', 'subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_6'])
dict_keys(['experiment_0', 'experiment_1'])
dict_keys(['stimuli_11', 'stimuli_12'])
(2000, 46)


In [12]:
os.makedirs("intermediate", exist_ok=True)
with open("intermediate/default_processed.pickle", "wb") as f:
    pickle.dump(Vs, f)

In [4]:
with open("intermediate/default_processed.pickle", "rb") as f:
    Vs = pickle.load(f)

#### map electrodes

In [5]:
# list of electrode locations
locs = {}
for subject in trange(len(alldat)):
    # assuming same location for 2 experiments within single subject
    loc = alldat[subject][0]["locs"]
    locs[f"subject_{subject}"] = loc
pprint(locs.keys())
pprint(locs[list(locs.keys())[0]].shape)
pprint(locs[list(locs.keys())[2]].shape)
pprint(locs[list(locs.keys())[6]].shape)

100%|██████████| 7/7 [00:00<00:00, 97867.09it/s]

dict_keys(['subject_0', 'subject_1', 'subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_6'])
(46, 3)
(48, 3)
(64, 3)


In [6]:
# check # of electrodes per experiment
nums_electrodes = {}
for k1 in Vs.keys():
    for k2 in Vs[k1].keys():
        for k3 in Vs[k1][k2].keys():
            num_electrodes = Vs[k1][k2][k3].shape[1]
            nums_electrodes[k1] = num_electrodes#f"subject_{k1}"
pprint(list(set(nums_electrodes.values())))# 3 possible types

[64, 48, 46]


In [ ]:
'''
Rank electrodes for conversion to relative position
-> Create 64 clusters by 64-electrode subjects
-> Seeing a cluster as "electrode location", 46/48 electrodes should be mapped to 64 locations
'''

In [7]:
electrode_indices = electrode_mapper(locs, nums_electrodes)
pprint(electrode_indices.keys())
pprint(len(electrode_indices[list(electrode_indices.keys())[0]]))
pprint(electrode_indices[list(electrode_indices.keys())[0]])

Number of electrodes for indexing: 64
dict_keys(['subject_0', 'subject_1', 'subject_2', 'subject_3', 'subject_4', 'subject_5', 'subject_6'])
46
array([36, 34, 34,  7,  7,  7,  7,  7, 36, 17, 58,  7,  7,  7,  7, 36, 17,
       22, 58, 58,  7,  7, 27, 36, 31, 46,  3, 61, 15, 27, 15, 21, 31, 59,
       20, 62, 15, 15, 15,  9, 59, 20, 20, 62, 15, 15], dtype=int32)


In [ ]:
'''
Not much good for one-to-one mapping - let's pend it here
-> Perform EDAs within same num_electrodes
'''

#### from np to pd

In [8]:
# np to pd for pandas-profiling
dfs = {}
subjects, num_electrodes = np.array(list(nums_electrodes.keys())), np.array(list(nums_electrodes.values()))
for num in tqdm(list(set(list(nums_electrodes.values())))):
    Vs_same_num_electrode = {}
    ## first split source data with num_electrodes
    subjects2concat = subjects[num_electrodes==num]
    for subj in subjects2concat:
        Vs_same_num_electrode[subj] = Vs[subj]
    dfs[num] = dat_df_generator(Vs_same_num_electrode)
# pprint(dfs.keys())
# pprint(dfs[list(dfs.keys())[0]].columns)
df = dfs[list(dfs.keys())[2]]
df[(df.subject=="subject_0")&(df.experiment=="experiment_0")&(df.stimulus=="stimuli_11")].head()

100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


,t,subject,experiment,stimulus,electrode_0,electrode_1,electrode_2,electrode_3,electrode_4,electrode_5,...,electrode_36,electrode_37,electrode_38,electrode_39,electrode_40,electrode_41,electrode_42,electrode_43,electrode_44,electrode_45
0,0,subject_0,experiment_0,stimuli_11,1.239507,1.024637,1.172002,1.237944,1.341469,1.088809,...,1.141752,1.169815,0.972612,1.103905,1.343518,1.162155,0.930232,1.020301,1.073476,1.249586
1,1,subject_0,experiment_0,stimuli_11,1.239031,1.026944,1.173505,1.237956,1.338178,1.090244,...,1.136742,1.170247,0.972295,1.105326,1.345350,1.162852,0.927191,1.020325,1.069490,1.249350
2,2,subject_0,experiment_0,stimuli_11,1.238430,1.029383,1.175010,1.238074,1.334629,1.091635,...,1.131631,1.170363,0.971934,1.106739,1.346978,1.163303,0.924028,1.020430,1.065542,1.248698
3,3,subject_0,experiment_0,stimuli_11,1.237704,1.031949,1.176508,1.238295,1.330826,1.092976,...,1.126424,1.170165,0.971533,1.108135,1.348391,1.163506,0.920750,1.020611,1.061643,1.247623
4,4,subject_0,experiment_0,stimuli_11,1.236857,1.034638,1.177991,1.238617,1.326777,1.094265,...,1.121128,1.169655,0.971095,1.109508,1.349577,1.163463,0.917363,1.020863,1.057801,1.246119


### EDA

#### plotly

In [15]:
df[electrode_cols_batched[0]].head()

,electrode_0,electrode_1,electrode_2,electrode_3,electrode_4
0,0.979850,0.981365,1.056163,0.982663,0.924497
1,0.979837,0.981416,1.058846,0.982693,0.924555
2,0.979821,0.981463,1.061517,0.982719,0.924658
3,0.979802,0.981508,1.064168,0.982742,0.924807
4,0.979779,0.981550,1.066794,0.982761,0.925000


In [19]:
# note: showing only 9 plots, since it looks like too many plots in single stdout go wrong
subject, experiment = "subject_1", "experiment_0"
tgt_electrodes = [
    "electrode_0","electrode_3","electrode_6",
    "electrode_9","electrode_12","electrode_15",
    "electrode_18","electrode_21","electrode_24"]
df = dfs[64].copy()
# electrode_cols_batched = convert_to_batches(df.columns[df.columns.str.contains("electrode")], batch_num=3)
electrode_cols_batched = convert_to_batches(tgt_electrodes, batch_num=3)

df = df[(df["subject"]==subject)&(df["experiment"]==experiment)]
for batch in electrode_cols_batched:
    for col in batch:
        df_tmp = df[["t", "stimulus", col]]
        df11 = df_tmp[df_tmp["stimulus"]=="stimuli_11"].rename({col: "stimuli_11"}, axis=1).drop("stimulus", axis=1)
        df12 = df_tmp[df_tmp["stimulus"]=="stimuli_12"].rename({col: "stimuli_12"}, axis=1).drop("stimulus", axis=1)
        df_tmp = df11.merge(df12.set_index("t"), left_on="t", right_index=True)
        df_tmp["electrode"] = col
        if col==batch[0]:
            df_plot = df_tmp.copy()
        else:
            df_plot = pd.concat([df_plot, df_tmp])
    es = [b.split("_")[-1] for b in batch]
    fig = px.line(
        df_plot, x="t", y=["stimuli_11", "stimuli_12"],
        title=f'{subject}_{experiment}_e_{"_".join(es)}', facet_col="electrode"
        )
    fig.show()


#### pdp

In [21]:
shared_cols = ["t", "subject", "experiment", "stimulus"]
electrode_cols = [col for col in dfs[64].columns if col not in shared_cols]
electrode_cols_batched = convert_to_batches(electrode_cols)
cols_batched = [shared_cols+cols for cols in electrode_cols_batched]
pprint(cols_batched[0])

['t',
 'subject',
 'experiment',
 'stimulus',
 'electrode_0',
 'electrode_1',
 'electrode_2',
 'electrode_3',
 'electrode_4']


In [26]:
os.makedirs(os.path.join("intermediate", "pdp"), exist_ok=True)
for cols,es in tqdm(zip(cols_batched,electrode_cols_batched)):
    profile = pdp.ProfileReport(dfs[64][cols], config_file="processing/pdp_config.yaml")
    e_nums = [e.split("_")[-1] for e in es]
    profile.to_file(os.path.join("intermediate", "pdp", f"e{min(e_nums)}_{max(e_nums)}.html"))


0it [00:00, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

13it [02:30, 11.57s/it]
